In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif' #解决一些字体显示乱码问题
import warnings; warnings.simplefilter('ignore') #忽略警告信息
import numpy as np
import pandas as pd
import tushare as ts

#数据获取与处理
data = ts.get_k_data('hs300', start = '2010-01-01', end='2017-06-30')    
data.rename(columns={'close': 'price'}, inplace=True)
data.set_index('date', inplace = True)
data['SMA_10'] = data['price'].rolling(10).mean()
data['SMA_60'] = data['price'].rolling(60).mean()
data[['price','SMA_10','SMA_60']].plot(title='HS300 stock price | 10 & 60 days SMAs', figsize=(10, 6))
data['10-60'] = data['SMA_10'] - data['SMA_60'] #震荡市金叉死叉较多，会频繁触发买卖信号，为了防止假突破(金叉后马上死叉)，要等金叉或死叉超过长期均线一定阈值再做多或做空
SD = 20 #阈值
data['regime'] = np.where(data['10-60'] > SD, 1, 0) #1开多仓，-1开空仓，0平仓
data['regime'] = np.where(data['10-60'] < -SD, -1, data['regime']) #注意这里为False值不能写0，不然会将1全部覆盖为0
data['regime'].value_counts()
data.dropna(inplace=True) #去掉NaN空值
data['regime'].plot(ylim=[-1.1, 1.1], title='Market Positioning') #画position图

data['returns'] = np.log(data['price'] / data['price'].shift(1))
# data['returns_dis'] = data['price']/data['price'].shift(1)-1 #离散计算return方法1
# data['return_dis2'] = data['price'].pct_change() #离散计算return方法2
data['returns'].hist(bins=35)
data['strategy'] = data['regime'].shift(1) * data['returns'] #由于今天的收益率、量化条件是否满足、是否持有头寸都是今天收盘才能知道，只有明天才能实际进行买卖操作，所以要将position往后移一位进行收益计算(一定要注意不要有未来函数问题，会高估利润)
data[['returns', 'strategy']].sum()
data[['returns', 'strategy']].cumsum().apply(np.exp).plot(figsize=(10, 6))   #可视化累计收益以判断策略是否能跑赢长期持有

#策略收益风险评估
data[['returns', 'strategy']].mean()*252 #年化收益率
data[['returns', 'strategy']].std()*252**0.5 #年化风险(标准差乘√252)
data['cumret'] = data['strategy'].cumsum().apply(np.exp) #策略的累计收益
data['cummax'] = data['cumret'].cummax() #用.cummax()而不用max()是因为未来的最大值不能用来算之前的回撤，只能用当期的历史最大值算回撤
data[['cumret', 'cummax']].plot(figsize=(10, 6))
drawdown = (data['cummax'] - data['cumret']) #累计收益的最大值与累积收益的差值
drawdown.max() #最大回撤(累计收益最大值跌到最小值的下跌幅度，表征可能的最大损失。实务中回撤10%就已经很大了，一般5%-6%)
temp=drawdown[drawdown==0] #提取当期为历史最大值的行
periods=(temp.index[1:].to_datetime() - temp.index[:-1].to_datetime()) #计算回撤从开始下跌到涨回来的最长时间是多久(创新高由于天数少不会)，这里是切片第二个到最后一个数据index减第一个到倒数第二个数据的index从而算出每次drawdown==0的时间间隔
period.max() #取最长时间